## ***<center>KPI's</center>***

**<center>En el presente notebook, desarrollaremos los KPI's solicitados y metricas (en caso de ser necesario) para desarrollo de los mismos</center>**

***

In [50]:
# importacion de librerias necesarias: 
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

In [51]:
# lectura de los archivos: 
homicidios = pd.read_csv(r'Data/homicidios_fatal.csv')
lesiones = pd.read_csv(r'Data/lesiones_df.csv')

In [52]:
homicidios['fecha'] = pd.to_datetime(homicidios['fecha'])
lesiones['fecha_y'] = pd.to_datetime(lesiones['fecha_y'])

* ***concatenaremos ambos DataFrame realizando previas transformaciones en los nombres de las columnas para poder calcular los KPI's***

In [53]:
print('columnas homicidios: ', homicidios.columns)
print('columnas lesiones: ', lesiones.columns)

columnas homicidios:  Index(['id_hecho', 'n_victimas', 'fecha', 'franja_horaria',
       'direccion_normalizada', 'comuna', 'longitud', 'latitud', 'victima',
       'acusado', 'rol', 'sexo', 'edad', 'gravedad', 'dia_semana'],
      dtype='object')
columnas lesiones:  Index(['id_hecho', 'n_victimas', 'franja_hora', 'comuna', 'tipo_calle',
       'otra_direccion', 'longitud', 'latitud', 'victima', 'acusado',
       'gravedad', 'fecha_y', 'sexo', 'edad'],
      dtype='object')


In [54]:
lesiones.rename(columns= {'franja_hora' : 'franja_horaria', 'fecha_y': 'fecha', 'otra_direccion': 'direccion_normalizada'}, inplace= True)

In [55]:
siniestros = pd.concat([homicidios, lesiones])

In [56]:
siniestros.to_csv('Data/siniestros.csv', index=False)

In [57]:
siniestros.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28315 entries, 0 to 27597
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id_hecho               28315 non-null  object        
 1   n_victimas             28315 non-null  int64         
 2   fecha                  28315 non-null  datetime64[ns]
 3   franja_horaria         28307 non-null  float64       
 4   direccion_normalizada  7177 non-null   object        
 5   comuna                 27201 non-null  object        
 6   longitud               26659 non-null  object        
 7   latitud                26659 non-null  object        
 8   victima                15484 non-null  object        
 9   acusado                10374 non-null  object        
 10  rol                    706 non-null    object        
 11  sexo                   26081 non-null  object        
 12  edad                   23791 non-null  float64       
 13  graved

In [58]:
siniestros['gravedad'].replace({'GRAVE': 'grave', 'FATAL': 'fatal'}, inplace=True)


In [59]:
siniestros['gravedad'].unique()

array(['fatal', 'lesionado', 'leve', 'grave'], dtype=object)

## **1)**

*  **Reducir en un 10% la tasa de homicidios en siniestros viales de los últimos seis meses, en CABA, en comparación con la tasa de homicidios en siniestros viales del semestre anterior:**
    - Definimos a la tasa de homicidios en siniestros viales como el número de víctimas fatales en accidentes de tránsito por cada 100,000 habitantes en un área geográfica durante un período de tiempo específico. Su fórmula es: 

$$
\text{Tasa de homicidios en siniestros viales} = \left( \frac{\text{Número de homicidios en siniestros viales}}{\text{Población total}} \right) \times 100{,}000
$$


In [60]:
# creo una funcion para asignar el semestre 
def asignar_semestre(fecha):
    if fecha.month <= 6:
        return 1  # Primer semestre
    else:
        return 2  # Segundo semestre

In [61]:
# Agrego la columna 'semestre' utilizando la función anterior
siniestros['semestre'] = siniestros['fecha'].apply(asignar_semestre)

In [62]:
siniestros['anio'] = siniestros['fecha'].dt.year # extraigo el año de la columna fecha

In [63]:
# agrupo por año, semestre y magnitud de la gravedad creando un nuevo df para el kpi1
kpi1 = siniestros.groupby(['anio','semestre','gravedad']).size().reset_index(name= 'cantidad_homicidios')

In [64]:
kpi1 = kpi1[kpi1['gravedad'] == 'fatal']

In [65]:
kpi1['semestre_anterior'] = kpi1['cantidad_homicidios'].shift(periods=1, fill_value=0)

In [66]:
kpi1['Tasa Actual'] = (kpi1['cantidad_homicidios']/3121707) * 100000
kpi1['Tasa Anterior'] = (kpi1['semestre_anterior']/3121707) * 100000

In [68]:
kpi1['variacion%'] = (kpi1['cantidad_homicidios'] - kpi1['semestre_anterior']) / kpi1['semestre_anterior'] * 100
kpi1['variacion%'] = (kpi1['Tasa Actual'] - kpi1['Tasa Anterior']) / kpi1['Tasa Anterior'] * 100

In [69]:
kpi1 

,anio,semestre,gravedad,cantidad_homicidios,semestre_anterior,Tasa Actual,Tasa Anterior,variacion%
0,2016,1,fatal,46,0,1.473553,0.000000,inf
2,2016,2,fatal,38,46,1.217283,1.473553,-17.391304
4,2017,1,fatal,68,38,2.178295,1.217283,78.947368
6,2017,2,fatal,66,68,2.114228,2.178295,-2.941176
8,2018,1,fatal,70,66,2.242363,2.114228,6.060606
9,2018,2,fatal,79,70,2.530667,2.242363,12.857143
10,2019,1,fatal,57,79,1.825924,2.530667,-27.848101
13,2019,2,fatal,47,57,1.505587,1.825924,-17.543860
16,2020,1,fatal,32,47,1.025080,1.505587,-31.914894
19,2020,2,fatal,51,32,1.633722,1.025080,59.375000


dado que necesitamos el dato de la poblacion total, visitamos el sitio oficial de [INDEC](https://www.indec.gob.ar/indec/web/Nivel4-Tema-2-41-165) (Instituto Nacional de Estadisticas y Censos) cuadro 1.1 para encontrar los siguientes datos: 

<p align="center">
    <img src="images/poblacion.png">
</p>

como podemos observar, segun el censo en 2022, la poblacion es de un total de 3.121.707 en CABA y teniendo en cuenta que los censos son realizados el primer mes del año, consideramos dato acertivo la cantidad de habitantes para el año 2021.

- estos datos sera utilizados en nuestro calculo del KPI en Power Bi

In [70]:
kpi1.to_csv('kpi1.csv', index = False)

***

### **2)**

* **Reducir en un 7% la cantidad de accidentes mortales de motociclistas en el último año, en CABA, respecto al año anterior.**
    - Definimos a la cantidad de accidentes mortales de motociclistas en siniestros viales como el número absoluto de accidentes fatales en los que estuvieron involucradas víctimas que viajaban en moto en un determinado periodo temporal. Su fórmula para medir la evolución de los accidentes mortales con víctimas en moto es:

$$
\text{Δ accidentes mortales en moto} = \left( \frac{\text{N° accidentes mortales en moto en el año anterior} - \text{N° accidentes mortales en moto en el año actual}}{\text{N° accidentes mortales en moto en el año anterior}} \right) \times 100
$$


In [52]:
siniestros.columns

Index(['id_hecho', 'n_victimas', 'fecha', 'franja_horaria',
       'direccion_normalizada', 'comuna', 'longitud', 'latitud', 'victima',
       'acusado', 'rol', 'sexo', 'edad', 'gravedad', 'dia_semana',
       'tipo_calle', 'semestre', 'anio'],
      dtype='object')

In [53]:
siniestros['victima'].unique() # conozco como estan escritos los datos en la columna victima

array(['MOTO', 'AUTO', 'PEATON', nan, 'CARGAS', 'BICICLETA', 'PASAJEROS',
       'MOVIL', 'OBJETO FIJO', 'PEATON_MOTO', 'CICLISTA',
       'TRANSPORTE PUBLICO', 'CAMIONETA', 'OTRO', 'CAMION', 'MONOPATIN',
       'UTILITARIO', 'TAXI', 'MIXTO'], dtype=object)

In [44]:
# Agrupo y creo un nuevo df para el kpi2
kpi2 = siniestros.groupby(['anio','victima','gravedad']).size().reset_index(name= 'cantidad_homicidios') 

In [45]:
kpi2 = kpi2[kpi2['gravedad'] == 'fatal']


In [47]:
#filtro por MOTO
filtro_moto = kpi2[kpi2['victima'] == 'MOTO']
kpi2 = filtro_moto.reset_index()


In [49]:
kpi2.drop(columns=['index'], inplace = True) #elimino esa columna de mas

In [72]:
kpi2['Anio anterior'] = kpi2['cantidad_homicidios'].shift(periods=1, fill_value=0)

In [73]:
kpi2['Tasa Actual'] = (kpi2['cantidad_homicidios']/3121707) * 100000
kpi2['Tasa Anterior'] = (kpi2['Anio anterior']/3121707) * 100000

In [74]:
kpi2['variacion%'] = (kpi2['cantidad_homicidios'] - kpi2['Anio anterior']) / kpi2['Anio anterior'] * 100
kpi2['variacion%'] = (kpi2['Tasa Actual'] - kpi2['Tasa Anterior']) / kpi2['Tasa Anterior'] * 100

In [75]:
kpi2

,anio,victima,gravedad,cantidad_homicidios,Anio anterior,Tasa Actual,Tasa Anterior,variacion%
0,2016,MOTO,fatal,37,0,1.185249,0.000000,inf
1,2017,MOTO,fatal,54,37,1.729823,1.185249,45.945946
2,2018,MOTO,fatal,57,54,1.825924,1.729823,5.555556
3,2019,MOTO,fatal,50,57,1.601688,1.825924,-12.280702
4,2020,MOTO,fatal,30,50,0.961013,1.601688,-40.000000
5,2021,MOTO,fatal,46,30,1.473553,0.961013,53.333333


* exporto para posterior uso:

In [76]:
kpi2.to_csv('kpi2.csv', index = False)

***

## **3)**

*  **Reducir un 10% la tasa de mortalidad de la comuna 1 respecto al año anterior**
    - se calcula de la siguiente manera: 
$$
\text{Δ  mortalidad anual} = \left( \frac{\text{N° accidentes mortales en el año anterior comuna 1} - \text{N° accidentes mortales en el año actual comuna 1}}{\text{N° accidentes mortales en el año anterior comuna 1}} \right) \times 100
$$



* para este kpi necesitamos trabajar con los datos de comuna. A continuacion procedemos a tratar estos para su uso.

In [59]:
siniestros['comuna'].unique()

array([8, 9, 1, 11, 15, 4, 7, 12, 3, 13, 14, 10, 6, 2, 5, '14', '8', '7',
       '3', '15', '11', '9', '1', '4', '12', '2', '5', '6', '13', '10',
       'No Especificada', nan], dtype=object)

Como podemos observar, hay varios datos que son tipo str, o nan.
- Para los datos 'No Especificada' los llevaremos a valor de comuna =  0 al igual que los datos nan.

Esto a fines de poder trabajar unicamente con enteros 

In [77]:
siniestros['comuna'] = siniestros['comuna'].replace(['No Especificada', np.nan], 0)

In [78]:
siniestros['comuna'] = siniestros['comuna'].astype(int)

In [79]:
kpi3 = siniestros.groupby(['anio','gravedad','comuna']).size().reset_index(name= 'cantidad_homicidios')
filtros = kpi3.loc[(kpi3['gravedad'] == 'fatal') & (kpi3['comuna'] == 1)]


In [80]:
kpi3 = filtros.reset_index()

In [81]:
kpi3.drop(columns=['index'], inplace = True)

In [82]:
kpi3['Anio anterior'] = kpi3['cantidad_homicidios'].shift(periods=1, fill_value=0)

In [83]:
kpi3['Tasa Actual'] = (kpi3['cantidad_homicidios']/3121707) * 100000
kpi3['Tasa Anterior'] = (kpi3['Anio anterior']/3121707) * 100000

In [84]:
kpi3['variacion%'] = (kpi3['cantidad_homicidios'] - kpi3['Anio anterior']) / kpi3['Anio anterior'] * 100
kpi3['variacion%'] = (kpi3['Tasa Actual'] - kpi3['Tasa Anterior']) / kpi3['Tasa Anterior'] * 100

In [85]:
kpi3

,anio,gravedad,comuna,cantidad_homicidios,Anio anterior,Tasa Actual,Tasa Anterior,variacion%
0,2016,fatal,1,17,0,0.544574,0.000000,inf
1,2017,fatal,1,20,17,0.640675,0.544574,17.647059
2,2018,fatal,1,20,20,0.640675,0.640675,0.000000
3,2019,fatal,1,12,20,0.384405,0.640675,-40.000000
4,2020,fatal,1,10,12,0.320338,0.384405,-16.666667
5,2021,fatal,1,9,10,0.288304,0.320338,-10.000000


In [86]:
kpi3.to_csv('kpi3.csv', index = False)